In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%matplotlib notebook

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import os
import json
from collections import Counter
from sklearn.cluster import DBSCAN

In [ ]:
cur_dir = './drive/My Drive/DiffusionS1/'
runs_dir = cur_dir + "runs/"
para = json.load(open(cur_dir +  "/config.json"))

In [ ]:
he_events = 0
contained_events = 0
strack_events = 0
req_par = ['X', 'Y', 'Z', 'E']
for run in os.listdir(runs_dir):
    he_file = pd.HDFStore(cur_dir + "output_files/" + "he_events.h5")
    contained_file = pd.HDFStore(cur_dir + "/output_files/" + "contained_events.h5")
    strack_file = pd.HDFStore(cur_dir + "/output_files/" + "strack_events.h5")
    map_dir = runs_dir + str(run) + "/map/"
    run_map = pd.read_hdf(map_dir + os.listdir(map_dir)[0], "/e0")
    data_dir = runs_dir + str(run) + "/data/"
    for data_file in os.listdir(data_dir):
        data = pd.read_hdf(data_dir + data_file, "/RECO/Events")
        conversion = np.nanmax(run_map[:len(run_map)+1])
        for event in data.event.unique():
            my_event = data[data.event == event]
            my_event = my_event[req_par]
            pe = np.nansum(my_event.E)
            # the original files are in pe, converting them to MeV here
            energy = ((pe*41.5)/conversion)/1000
            # selecting only the high energy events
            if energy >= para["high_energy_thres"]:
                he_events += 1
                he_file.put("/RECO/Events", my_event, format='t', append=True, data_columns=True)
                my_event_z = my_event[(my_event.Z > para["z_thres_lower"]) & (my_event.Z < para["z_thres_upper"])]
                radius = np.sqrt((my_event.X**2 + my_event.Y**2))
                # selecting only the contained events
                if all(radius < para["radius_thres"]) and len(my_event_z) == len(my_event):
                    contained_events += 1
                    contained_file.put("/RECO/Events", my_event, format='t', append=True, data_columns=True)
                    image_3d = np.stack((my_event.X, my_event.Y, my_event.Z), axis=1)
                    db = DBSCAN(eps=20, min_samples=30).fit(image_3d)
                    labels = db.labels_
                    # if the event only has a single cluster and no background OR
                    # if the event has two clusters and one of those is just the background
                    # put the event in the single track file, else, move on to next event
                    if len(np.unique(labels)) == 1 and np.unique(labels)[0] != -1:
                        pass
                    elif len(np.unique(labels)) == 2 and -1 in np.unique(labels):
                        pass
                    else:
                        continue
                    strack_events += 1
                    strack_file.put("/RECO/Events", my_event, format='t', append=True, data_columns=True)

In [ ]:
print(he_events, contained_events, strack_events)